In [4]:
from nba_api.stats.endpoints import ShotChartDetail
import json
import pandas as pd
import time
import os

# Function to fetch shot data with retries
def fetch_shot_data_with_retries(
    seasons, season_types, team_id=0, player_id=0, context_measure='FGA', max_retries=3, output_file='shot_data.csv'
):
    all_data = []
    for season in seasons:
        for season_type in season_types:
            retries = 0
            while retries < max_retries:
                try:
                    print(f"Fetching data for {season} - {season_type}")
                    response = ShotChartDetail(
                        team_id=team_id,
                        player_id=player_id,
                        season_nullable=season,
                        season_type_all_star=season_type,
                        context_measure_simple=context_measure
                    )
                    content = json.loads(response.get_json())
                    results = content['resultSets'][0]
                    headers = results['headers']
                    rows = results['rowSet']
                    df = pd.DataFrame(rows, columns=headers)
                    df['SEASON_TYPE'] = season_type
                    all_data.append(df)
                    
                    # Save to CSV periodically
                    save_data_to_csv(df, output_file)
                    
                    # Add a delay to avoid overwhelming the server
                    time.sleep(1)
                    break
                except TimeoutError:
                    retries += 1
                    print(f"Timeout. Retry {retries}/{max_retries} for {season} - {season_type}")
                    time.sleep(2)  # Wait before retrying
                except Exception as e:
                    print(f"Error for {season} - {season_type}: {e}")
                    break
    
    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()

# Function to save DataFrame to CSV
def save_data_to_csv(data, filename='shot_data.csv'):
    if not os.path.exists(filename):
        data.to_csv(filename, index=False)
    else:
        data.to_csv(filename, mode='a', header=False, index=False)

# Define seasons and types
seasons = [f"{year}-{str(year+1)[-2:]}" for year in range(2021, 2023)]
season_types = ['Regular Season', 'Playoffs']

# Fetch and process data
output_file = 'shot_data.csv'
if os.path.exists(output_file):
    os.remove(output_file)  # Remove existing file to avoid duplicates

shot_data = fetch_shot_data_with_retries(seasons, season_types, team_id=0, player_id=0, context_measure='FGA', output_file=output_file)

# Display the first few rows of the final data
if not shot_data.empty:
    print(shot_data.head())
else:
    print("No data fetched.")


Fetching data for 2021-22 - Regular Season
Fetching data for 2021-22 - Playoffs
Fetching data for 2022-23 - Regular Season
Fetching data for 2022-23 - Playoffs
           GRID_TYPE     GAME_ID  GAME_EVENT_ID  PLAYER_ID  \
0  Shot Chart Detail  0022100001              7    1628960   
1  Shot Chart Detail  0022100001             15     203507   
2  Shot Chart Detail  0022100001             18     203507   
3  Shot Chart Detail  0022100001             20    1629651   
4  Shot Chart Detail  0022100001             22     203507   

             PLAYER_NAME     TEAM_ID        TEAM_NAME  PERIOD  \
0          Grayson Allen  1610612749  Milwaukee Bucks       1   
1  Giannis Antetokounmpo  1610612749  Milwaukee Bucks       1   
2  Giannis Antetokounmpo  1610612749  Milwaukee Bucks       1   
3            Nic Claxton  1610612751    Brooklyn Nets       1   
4  Giannis Antetokounmpo  1610612749  Milwaukee Bucks       1   

   MINUTES_REMAINING  SECONDS_REMAINING  ...  SHOT_ZONE_RANGE SHOT_DISTANCE 